# Data Preprocessing - Movies Revenue Prediction

Tiền xử lý dữ liệu để chuẩn bị cho mô hình học máy dự đoán doanh thu phim

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Tải dữ liệu và kiểm tra dữ liệu ban đầu

In [11]:
# Tải dữ liệu
df = pd.read_csv('data/raw/movies_dataset_revenue.csv')

print("="*80)
print("THÔNG TIN DỮ LIỆU BAN ĐẦU")
print("="*80)
print(f"\nKích thước dữ liệu: {df.shape}")
print(f"\nCác cột trong dữ liệu:")
print(df.columns.tolist())

print(f"\n\nThông tin dữ liệu:")
print(df.info())

print(f"\n\nSố lượng giá trị NULL:")
null_counts = df.isnull().sum()
null_percentage = (null_counts / len(df) * 100).round(2)
null_df = pd.DataFrame({'Column': df.columns, 'Null_Count': null_counts, 'Null_Percentage': null_percentage})
print(null_df[null_df['Null_Count'] > 0].sort_values('Null_Percentage', ascending=False))

print(f"\n\nDữ liệu đầu tiên:")
print(df.head())

# Lưu lại bản gốc để so sánh
df_original = df.copy()

THÔNG TIN DỮ LIỆU BAN ĐẦU

Kích thước dữ liệu: (11187, 17)

Các cột trong dữ liệu:
['id', 'title', 'release_date', 'budget', 'revenue', 'runtime', 'rating', 'vote_count', 'popularity', 'genres', 'production_companies', 'production_countries', 'director', 'cast', 'keywords', 'original_language', 'collection']


Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11187 entries, 0 to 11186
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    11187 non-null  int64  
 1   title                 11187 non-null  object 
 2   release_date          11187 non-null  object 
 3   budget                11187 non-null  int64  
 4   revenue               11187 non-null  int64  
 5   runtime               11187 non-null  int64  
 6   rating                11187 non-null  float64
 7   vote_count            11187 non-null  int64  
 8   popularity            11187 non-null  float64

## 2. Xử lý giá trị thiếu (Missing Values)

In [12]:
print("="*80)
print("XỬ LÝ GIÁ TRỊ THIẾU")
print("="*80)

# Chuyển đổi release_date sang datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['year'] = df['release_date'].dt.year
df['month'] = df['release_date'].dt.month
df['quarter'] = df['release_date'].dt.quarter

# Xác định các cột quantitative và categorical
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nCác cột số: {numerical_cols}")
print(f"Các cột phân loại: {categorical_cols}")

# Xử lý missing values cho các cột số - dùng median
print(f"\nXử lý missing values cho cột số:")
for col in numerical_cols:
    if df[col].isnull().sum() > 0:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)
        print(f"  - {col}: Điền median = {median_val:.2f}")

# Xử lý missing values cho các cột categorical - dùng mode hoặc 'Unknown'
print(f"\nXử lý missing values cho cột phân loại:")
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode_val = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
        df[col].fillna(mode_val, inplace=True)
        print(f"  - {col}: Điền mode/Unknown = {mode_val}")

# Kiểm tra sau khi xử lý
print(f"\nKiểm tra sau khi xử lý:")
print(f"Số lượng missing values còn lại: {df.isnull().sum().sum()}")

# Lọc dữ liệu với budget > 0 và revenue > 0
df = df[(df['budget'] > 0) & (df['revenue'] > 0)].copy()
print(f"\nSau khi lọc budget > 0 và revenue > 0: {len(df)} phim (giảm {len(df_original) - len(df)} phim)")

XỬ LÝ GIÁ TRỊ THIẾU

Các cột số: ['id', 'budget', 'revenue', 'runtime', 'rating', 'vote_count', 'popularity', 'year', 'month', 'quarter']
Các cột phân loại: ['title', 'genres', 'production_companies', 'production_countries', 'director', 'cast', 'keywords', 'original_language', 'collection']

Xử lý missing values cho cột số:

Xử lý missing values cho cột phân loại:
  - genres: Điền mode/Unknown = Drama
  - production_companies: Điền mode/Unknown = DreamWorks Animation
  - production_countries: Điền mode/Unknown = United States of America
  - cast: Điền mode/Unknown = Wasabi Mizuta, Megumi Oohara, Yumi Kakazu, Subaru Kimura, Tomokazu Seki
  - keywords: Điền mode/Unknown = woman director
  - collection: Điền mode/Unknown = Detective Conan Collection

Kiểm tra sau khi xử lý:
Số lượng missing values còn lại: 0

Sau khi lọc budget > 0 và revenue > 0: 6930 phim (giảm 4257 phim)


## 3. Xóa bản ghi trùng lặp và xử lý Outliers

In [13]:
print("="*80)
print("XỬ LÝ TRÙNG LẶP VÀ OUTLIERS")
print("="*80)

# Xóa bản ghi trùng lặp dựa trên cột 'id'
duplicate_count = df.duplicated(subset=['id']).sum()
print(f"\nSố bản ghi trùng lặp (dựa trên id): {duplicate_count}")
if duplicate_count > 0:
    df = df.drop_duplicates(subset=['id'], keep='first')
    print(f"Sau khi xóa: {len(df)} phim")

# Xử lý Outliers sử dụng IQR method cho các cột số
print(f"\nXác định Outliers sử dụng IQR method:")

outlier_indices = []
for col in ['budget', 'revenue', 'runtime', 'rating', 'vote_count', 'popularity']:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index.tolist()
        outlier_indices.extend(outliers)
        
        print(f"  - {col}: {len(outliers)} outliers (Range: [{lower_bound:.2f}, {upper_bound:.2f}])")

# Xóa outliers (tối đa 5% dữ liệu)
unique_outlier_indices = list(set(outlier_indices))
max_outliers_to_remove = int(len(df) * 0.05)

if len(unique_outlier_indices) > max_outliers_to_remove:
    print(f"\n⚠️ Số outliers vượt quá 5%, chỉ xóa {max_outliers_to_remove} outliers")
    unique_outlier_indices = unique_outlier_indices[:max_outliers_to_remove]
else:
    print(f"\nXóa {len(unique_outlier_indices)} outliers")
    df = df.drop(unique_outlier_indices)

print(f"Sau khi xử lý outliers: {len(df)} phim")

XỬ LÝ TRÙNG LẶP VÀ OUTLIERS

Số bản ghi trùng lặp (dựa trên id): 0

Xác định Outliers sử dụng IQR method:
  - budget: 656 outliers (Range: [-41250000.00, 80750000.00])
  - revenue: 798 outliers (Range: [-104464871.88, 182641453.12])
  - runtime: 214 outliers (Range: [56.00, 160.00])
  - rating: 124 outliers (Range: [4.20, 8.63])
  - vote_count: 718 outliers (Range: [-3008.12, 5426.88])
  - popularity: 289 outliers (Range: [-3.20, 11.01])

⚠️ Số outliers vượt quá 5%, chỉ xóa 346 outliers
Sau khi xử lý outliers: 6930 phim


## 4. Feature Engineering - Tạo các đặc trưng mới

In [14]:
print("="*80)
print("FEATURE ENGINEERING")
print("="*80)

# 1. Tính ROI (Return on Investment)
df['ROI'] = (df['revenue'] - df['budget']) / df['budget'] * 100
print(f"\n✓ Tạo đặc trưng ROI")

# 2. Tính Profit Margin
df['profit_margin'] = (df['revenue'] - df['budget']) / df['revenue'] * 100
print(f"✓ Tạo đặc trưng profit_margin")

# 3. Tính Profit
df['profit'] = df['revenue'] - df['budget']
print(f"✓ Tạo đặc trưng profit")

# 4. Phân loại Budget
df['budget_category'] = pd.cut(df['budget'], 
                               bins=[0, 10e6, 50e6, 150e6, float('inf')],
                               labels=['Low', 'Medium', 'High', 'VeryHigh'])
print(f"✓ Tạo đặc trưng budget_category")

# 5. Phân loại Revenue
df['revenue_category'] = pd.cut(df['revenue'],
                                bins=[0, 50e6, 200e6, 500e6, float('inf')],
                                labels=['Low', 'Medium', 'High', 'VeryHigh'])
print(f"✓ Tạo đặc trưng revenue_category")

# 6. Log transformation cho budget và revenue (giúp xử lý skewed distribution)
df['log_budget'] = np.log1p(df['budget'])
df['log_revenue'] = np.log1p(df['revenue'])
print(f"✓ Tạo đặc trưng log_budget và log_revenue")

# 7. Tính Rating per Vote
df['rating_per_vote'] = df['rating'] / (df['vote_count'] + 1)
print(f"✓ Tạo đặc trưng rating_per_vote")

# 8. Tính Popularity Score
df['popularity_score'] = df['popularity'] * df['rating']
print(f"✓ Tạo đặc trưng popularity_score")

# 9. Phân loại mùa từ tháng
def get_season(month):
    if pd.isna(month):
        return 'Unknown'
    elif month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df['season'] = df['month'].apply(get_season)
print(f"✓ Tạo đặc trưng season")

print(f"\nCác đặc trưng mới được tạo:")
print(df[['ROI', 'profit_margin', 'profit', 'budget_category', 'revenue_category', 
          'log_budget', 'log_revenue', 'rating_per_vote', 'popularity_score', 'season']].head())

FEATURE ENGINEERING

✓ Tạo đặc trưng ROI
✓ Tạo đặc trưng profit_margin
✓ Tạo đặc trưng profit
✓ Tạo đặc trưng budget_category
✓ Tạo đặc trưng revenue_category
✓ Tạo đặc trưng log_budget và log_revenue
✓ Tạo đặc trưng rating_per_vote
✓ Tạo đặc trưng popularity_score
✓ Tạo đặc trưng season

Các đặc trưng mới được tạo:
          ROI  profit_margin     profit budget_category revenue_category  \
0  377.369047      79.051847  339632142            High             High   
1  177.841618      64.008272  226748063            High             High   
2  351.957522      77.874027  362516248            High             High   
3  181.138211      64.430307  222800000            High             High   
4  434.428571      81.288426  304100000            High             High   

   log_budget  log_revenue  rating_per_vote  popularity_score  season  
0   18.315320    19.878440         0.000643         54.038236  Winter  
1   18.663627    19.685508         0.002461         44.104709  Spring  
2   18.45

## 5. Mã hóa biến phân loại (Encoding Categorical Variables)

In [15]:
print("="*80)
print("MÃ HÓA BIẾN PHÂN LOẠI")
print("="*80)

# Lưu bản sao để xử lý
df_processed = df.copy()

# 1. One-Hot Encoding cho budget_category, revenue_category, season
categorical_features_to_encode = ['budget_category', 'revenue_category', 'season']

print(f"\nOne-Hot Encoding cho các biến:")
for col in categorical_features_to_encode:
    dummies = pd.get_dummies(df_processed[col], prefix=col, drop_first=True)
    df_processed = pd.concat([df_processed, dummies], axis=1)
    print(f"  - {col}: {dummies.columns.tolist()}")

# 2. Mã hóa original_language (Label Encoding cho các ngôn ngữ phổ biến)
print(f"\nXử lý original_language:")
top_languages = df_processed['original_language'].value_counts().head(10).index.tolist()
df_processed['language_encoded'] = df_processed['original_language'].apply(
    lambda x: top_languages.index(x) if x in top_languages else -1
)
print(f"  - Top languages: {top_languages}")

# 3. Xử lý genres - tạo one-hot encoding cho top genres
print(f"\nXử lý genres:")
from collections import Counter

# Tách genres
genres_list_all = []
for genres_str in df_processed['genres'].dropna():
    if isinstance(genres_str, str):
        genres = [g.strip() for g in genres_str.split(',')]
        genres_list_all.extend(genres)

genres_count = Counter(genres_list_all)
top_genres = [g for g, c in genres_count.most_common(10)]
print(f"  - Top 10 genres: {top_genres}")

# Tạo binary features cho mỗi top genre
for genre in top_genres:
    df_processed[f'genre_{genre.lower()}'] = df_processed['genres'].apply(
        lambda x: 1 if isinstance(x, str) and genre in x else 0
    )

print(f"  - Tạo {len(top_genres)} binary features cho genres")

print(f"\nCác biến phân loại đã mã hóa: OK")

MÃ HÓA BIẾN PHÂN LOẠI

One-Hot Encoding cho các biến:
  - budget_category: ['budget_category_Medium', 'budget_category_High', 'budget_category_VeryHigh']
  - revenue_category: ['revenue_category_Medium', 'revenue_category_High', 'revenue_category_VeryHigh']
  - season: ['season_Spring', 'season_Summer', 'season_Winter']

Xử lý original_language:
  - Top languages: ['en', 'fr', 'hi', 'ru', 'es', 'it', 'zh', 'ko', 'ja', 'ta']

Xử lý genres:
  - Top 10 genres: ['Drama', 'Comedy', 'Thriller', 'Action', 'Romance', 'Adventure', 'Crime', 'Horror', 'Family', 'Fantasy']
  - Tạo 10 binary features cho genres

Các biến phân loại đã mã hóa: OK


## 6. Chuẩn hóa và Scale các đặc trưng số (Normalization & Scaling)

In [16]:
print("="*80)
print("CHUẨN HÓA VÀ SCALE DỮ LIỆU")
print("="*80)

# Xác định các cột số cần scale
features_to_scale = ['budget', 'runtime', 'rating', 'vote_count', 'popularity',
                     'ROI', 'profit_margin', 'profit', 'log_budget', 'log_revenue',
                     'rating_per_vote', 'popularity_score', 'year', 'month', 'quarter',
                     'language_encoded']

# Lọc các cột tồn tại
features_to_scale = [col for col in features_to_scale if col in df_processed.columns]

print(f"\nCác đặc trưng sẽ được scale: {features_to_scale}")

# Sử dụng StandardScaler (Z-score normalization)
scaler = StandardScaler()
df_processed[features_to_scale] = scaler.fit_transform(df_processed[features_to_scale])

print(f"\n✓ Áp dụng StandardScaler cho {len(features_to_scale)} đặc trưng")

# Hiển thị thống kê sau scaling
print(f"\nThống kê sau scaling (một số cột):")
print(df_processed[features_to_scale[:5]].describe())

CHUẨN HÓA VÀ SCALE DỮ LIỆU

Các đặc trưng sẽ được scale: ['budget', 'runtime', 'rating', 'vote_count', 'popularity', 'ROI', 'profit_margin', 'profit', 'log_budget', 'log_revenue', 'rating_per_vote', 'popularity_score', 'year', 'month', 'quarter', 'language_encoded']

✓ Áp dụng StandardScaler cho 16 đặc trưng

Thống kê sau scaling (một số cột):
             budget       runtime        rating    vote_count    popularity
count  6.930000e+03  6.930000e+03  6.930000e+03  6.930000e+03  6.930000e+03
mean   1.230377e-17  1.558478e-16 -6.069860e-16 -1.640503e-17 -4.101257e-18
std    1.000072e+00  1.000072e+00  1.000072e+00  1.000072e+00  1.000072e+00
min   -6.775849e-01 -5.151017e+00 -5.313562e+00 -5.665143e-01 -9.516815e-01
25%   -5.757375e-01 -7.159441e-01 -5.757515e-01 -5.261263e-01 -5.066737e-01
50%   -3.833592e-01 -2.024094e-01  5.477948e-02 -3.812865e-01 -1.739912e-01
75%    1.145613e-01  4.978653e-01  6.853105e-01  6.124099e-02  2.538429e-01
max    1.253768e+01  1.067519e+01  2.763158e+0

## 7. Chia dữ liệu thành Train và Test Sets

In [17]:
print("="*80)
print("CHIA DỮ LIỆU THÀNH TRAIN VÀ TEST SETS")
print("="*80)

# Chọn các đặc trưng cho mô hình
# Loại bỏ các cột không cần thiết (id, title, genres, dates, etc.)
columns_to_drop = ['id', 'title', 'release_date', 'genres', 'production_companies', 
                   'production_countries', 'director', 'cast', 'keywords', 'original_language',
                   'collection', 'budget_category', 'revenue_category', 'season']

# Lọc các cột tồn tại
columns_to_drop = [col for col in columns_to_drop if col in df_processed.columns]

# Các cột feature
X = df_processed.drop(columns=columns_to_drop + ['revenue'])
y = df_processed['revenue']

print(f"\nKích thước dữ liệu:")
print(f"  - X: {X.shape}")
print(f"  - y: {y.shape}")

print(f"\nCác feature sẽ được sử dụng:")
print(f"  {X.columns.tolist()}")

# Chia dữ liệu: 75% train, 25% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(f"\nSau khi chia dữ liệu:")
print(f"  - X_train: {X_train.shape}")
print(f"  - X_test: {X_test.shape}")
print(f"  - y_train: {y_train.shape}")
print(f"  - y_test: {y_test.shape}")

print(f"\nThống kê dữ liệu train:")
print(f"  - Doanh thu trung bình: ${y_train.mean():,.0f}")
print(f"  - Doanh thu max: ${y_train.max():,.0f}")
print(f"  - Doanh thu min: ${y_train.min():,.0f}")

print(f"\nThống kê dữ liệu test:")
print(f"  - Doanh thu trung bình: ${y_test.mean():,.0f}")
print(f"  - Doanh thu max: ${y_test.max():,.0f}")
print(f"  - Doanh thu min: ${y_test.min():,.0f}")

CHIA DỮ LIỆU THÀNH TRAIN VÀ TEST SETS

Kích thước dữ liệu:
  - X: (6930, 35)
  - y: (6930,)

Các feature sẽ được sử dụng:
  ['budget', 'runtime', 'rating', 'vote_count', 'popularity', 'year', 'month', 'quarter', 'ROI', 'profit_margin', 'profit', 'log_budget', 'log_revenue', 'rating_per_vote', 'popularity_score', 'budget_category_Medium', 'budget_category_High', 'budget_category_VeryHigh', 'revenue_category_Medium', 'revenue_category_High', 'revenue_category_VeryHigh', 'season_Spring', 'season_Summer', 'season_Winter', 'language_encoded', 'genre_drama', 'genre_comedy', 'genre_thriller', 'genre_action', 'genre_romance', 'genre_adventure', 'genre_crime', 'genre_horror', 'genre_family', 'genre_fantasy']

Sau khi chia dữ liệu:
  - X_train: (5197, 35)
  - X_test: (1733, 35)
  - y_train: (5197,)
  - y_test: (1733,)

Thống kê dữ liệu train:
  - Doanh thu trung bình: $80,161,447
  - Doanh thu max: $2,923,706,026
  - Doanh thu min: $1

Thống kê dữ liệu test:
  - Doanh thu trung bình: $83,592,452

## 8. Lưu dữ liệu đã tiền xử lý

In [18]:
print("="*80)
print("LƯU DỮ LIỆU ĐÃ TIỀN XỬ LÝ")
print("="*80)

# Tạo thư mục nếu chưa tồn tại
import os
os.makedirs('data/processed', exist_ok=True)

# Lưu các tập dữ liệu train/test
X_train.to_csv('data/processed/X_train.csv', index=False)
X_test.to_csv('data/processed/X_test.csv', index=False)
y_train.to_csv('data/processed/y_train.csv', index=False, header=['revenue'])
y_test.to_csv('data/processed/y_test.csv', index=False, header=['revenue'])

print(f"\n✓ Lưu X_train vào data/processed/X_train.csv")
print(f"✓ Lưu X_test vào data/processed/X_test.csv")
print(f"✓ Lưu y_train vào data/processed/y_train.csv")
print(f"✓ Lưu y_test vào data/processed/y_test.csv")

print(f"\n" + "="*80)
print(f"TÓM TẮT TIỀN XỬ LÝ DỮ LIỆU")
print(f"="*80)
print(f"\n📊 Dữ liệu gốc: {len(df_original)} phim")
print(f"📊 Dữ liệu sau xử lý: {len(df_processed)} phim")
print(f"📊 Tỷ lệ data còn lại: {len(df_processed)/len(df_original)*100:.1f}%")
print(f"\n✓ Train set: {len(X_train)} mẫu ({len(X_train)/(len(X_train)+len(X_test))*100:.1f}%)")
print(f"✓ Test set: {len(X_test)} mẫu ({len(X_test)/(len(X_train)+len(X_test))*100:.1f}%)")
print(f"✓ Số features: {X.shape[1]}")
print(f"\nCác feature được sử dụng:")
for i, feat in enumerate(X.columns, 1):
    print(f"  {i}. {feat}")
print(f"\n✓ Tất cả dữ liệu đã được lưu trong thư mục data/processed/")

LƯU DỮ LIỆU ĐÃ TIỀN XỬ LÝ

✓ Lưu X_train vào data/processed/X_train.csv
✓ Lưu X_test vào data/processed/X_test.csv
✓ Lưu y_train vào data/processed/y_train.csv
✓ Lưu y_test vào data/processed/y_test.csv

TÓM TẮT TIỀN XỬ LÝ DỮ LIỆU

📊 Dữ liệu gốc: 11187 phim
📊 Dữ liệu sau xử lý: 6930 phim
📊 Tỷ lệ data còn lại: 61.9%

✓ Train set: 5197 mẫu (75.0%)
✓ Test set: 1733 mẫu (25.0%)
✓ Số features: 35

Các feature được sử dụng:
  1. budget
  2. runtime
  3. rating
  4. vote_count
  5. popularity
  6. year
  7. month
  8. quarter
  9. ROI
  10. profit_margin
  11. profit
  12. log_budget
  13. log_revenue
  14. rating_per_vote
  15. popularity_score
  16. budget_category_Medium
  17. budget_category_High
  18. budget_category_VeryHigh
  19. revenue_category_Medium
  20. revenue_category_High
  21. revenue_category_VeryHigh
  22. season_Spring
  23. season_Summer
  24. season_Winter
  25. language_encoded
  26. genre_drama
  27. genre_comedy
  28. genre_thriller
  29. genre_action
  30. genre_roma